<a href="https://colab.research.google.com/github/Aaryan-N/UsbHub/blob/main/TrumpGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is my first time using python (The syntax is pretty easy as I know javascript and java) and my first time building an LLM! Thanks to this legend: https://www.youtube.com/watch?v=kCc8FmEb1nY&t=580s I somehow managed to turn a 2 hour tutorial into 5 hours RIP

Grab the Trump tweet data set from my repo. Also run this on a T4 setup for fast speed and just hit the run all under the runtime tab.

In [ ]:
!wget https://github.com/Aaryan-N/sentimentllm/raw/main/TrumpTwitterAllProper.txt

--2024-09-08 00:58:08--  https://github.com/Aaryan-N/sentimentllm/raw/main/TrumpTwitterAllProper.txt
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Aaryan-N/sentimentllm/main/TrumpTwitterAllProper.txt [following]
--2024-09-08 00:58:08--  https://raw.githubusercontent.com/Aaryan-N/sentimentllm/main/TrumpTwitterAllProper.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3367981 (3.2M) [text/plain]
Saving to: ‘TrumpTwitterAllProper.txt.3’

TrumpTwitterAllProp 100%[===================>]   3.21M  --.-KB/s    in 0.03s   

2024-09-08 00:58:08 (108 MB/s) - ‘TrumpTwitterAllProper.txt.3’ saved [3367981

Set variables

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
batch_size = 16 # Amount of tokens in a batch
block_size = 34 # Amount of batchs in a block as per my understanding
max_iters = 30000 # Increase me to get a better result
eval_interval = 100 # How often you want the program to print out the train loss and val loss
l_rate = 1e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

Open up the file and print the length of the text


In [ ]:
with open('TrumpTwitterAllProper.txt', 'r', encoding='utf-8') as f:
    text = f.read()
len(text)

3325571

Check if Colab has managed to read it

In [ ]:
print(text[:1000])

Tweet
"If the press would cover me accurately & honorably, I would have far less reason to ""tweet."" Sadly, I don't know if that will ever happen!"
I am thrilled to nominate Dr. @RealBenCarson as our next Secretary of the US Dept. of Housing and Urban Development… https://t.co/OJKuDFhP3r
their country (the U.S. doesn't tax them) or to build a massive military complex in the middle of the South China Sea? I don't think so!
"Did China ask us if it was OK to devalue their currency (making it hard for our companies to compete), heavily tax our products going into.."
".@FoxNews will be re-running ""Objectified: Donald Trump,"" the ratings hit produced by the great Harvey Levin of TMZ, at 8:00 P.M. Enjoy!"
The Green Party just dropped its recount suit in Pennsylvania and is losing votes in Wisconsin recount. Just a Stein scam to raise money!
expensive mistake! THE UNITED STATES IS OPEN FOR BUSINESS
"these companies are able to move between all 50 states, with no tax or tariff being charged.

Define encoding and decoding functions and characters

In [ ]:
chars = sorted(list(set(text))) # Create a list of the text and then sort it
vocab_size = len(chars) # Length of the sorted list
stoi = { ch:i for i,ch in enumerate(chars) } # Step to iterate over the characters
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # Encoder Function
decode = lambda l: ''.join([itos[i] for i in l]) # Decoder Function

Add torch data tensors so Torch can then use it

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long) # Pass encoded (tokenized) data to the tensor function

Splitting data for training and blocking

In [ ]:
train_val = int(0.9*len(data)) # 90% of this will be used for training and 10% used to ensure it isn't just copying the data set
train_data = data[:train_val]
val_data = data[train_val:]
train_data[:block_size+1]

tensor([53, 86, 68, 68, 83,  0,  3, 42, 69,  1, 83, 71, 68,  1, 79, 81, 68, 82,
        82,  1, 86, 78, 84, 75, 67,  1, 66, 78, 85, 68, 81,  1, 76, 68,  1])

Blocking now (Makes a lot of sense after he explained it. I'm new to python and machine learning in general so very exciting stuff!)

In [ ]:
torch.manual_seed(1432)

# So this basically splits the data into blocks so we don't just load the entire dataset onto the transformer because it would be very hardware intensive

def get_batch(split):
    data = train_data if split == 'train' else val_data #  If training then move into training sequence
    ix = torch.randint(len(data) - block_size, (batch_size,)) # Iterate through all the data in blocks as defined above
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

print('Success!')

Success!


Training the model

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # Call the training function
    return out # The output

Heads of self-attention (Nodes talking to each other)

In [ ]:
class Head(nn.Module):
    # Single head / Pretty chill and understandable
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False) # These functions multiply the matrix's generated
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) # Triangulate the matrix so there is a diagonal of only zeros

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (Block Size, Time, Context)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T) / As we cannnot multiply these we must transpose the matrix
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # Convert all zeros in the diagonal into negative infinity
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # Aggregation of the values
        v = self.value(x)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

# After this comment, it is really difficult for me to understand this he didn't really cover this in his video. I haven't even learnt all this in school :(
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

Adding bigram (This is probably the only part I have close to no clue how it works, so much math. How are we putting gradients on these numbers?) Also bigram isn't a model, it's just a name of this kind of model

In [ ]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1) # Apply softmax
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

# Number of parameters in the TrumpGPT model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters.', 'We not making ChatGPT with this one')

0.216436 M parameters. We not making ChatGPT with this one


Training the model and printing the loss value (More iterations means lower loss and train values aka better result. Redefine the iterations in the declare variables code block at the top) 20000 iterations took me about 17 minutes so keep this in mind!

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=l_rate) # Use the AdamW optimizer

for iter in range(max_iters):

    # Evaluate the loss on training and value sets occasionally and print
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"Step {iter}: Training loss {losses['train']:.5f}, Value loss {losses['val']:.5f}")
    # We want these values to be as low as possible meaning the model is losing less data and making less mistakes

    # Sample a batch of data
    xb, yb = get_batch('train')

    # Evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


Step 0: Training loss 4.99368, Value loss 4.99021
Step 100: Training loss 3.87605, Value loss 3.84083
Step 200: Training loss 3.58624, Value loss 3.54354
Step 300: Training loss 3.44978, Value loss 3.43299
Step 400: Training loss 3.31072, Value loss 3.32000
Step 500: Training loss 3.18375, Value loss 3.22486
Step 600: Training loss 3.08875, Value loss 3.15033
Step 700: Training loss 3.01187, Value loss 3.08618
Step 800: Training loss 2.95984, Value loss 3.04529
Step 900: Training loss 2.91341, Value loss 2.99445
Step 1000: Training loss 2.88269, Value loss 2.97105
Step 1100: Training loss 2.84081, Value loss 2.94508
Step 1200: Training loss 2.81898, Value loss 2.92351
Step 1300: Training loss 2.79155, Value loss 2.89683
Step 1400: Training loss 2.76422, Value loss 2.88668
Step 1500: Training loss 2.74400, Value loss 2.86644
Step 1600: Training loss 2.72783, Value loss 2.84873
Step 1700: Training loss 2.71071, Value loss 2.82417
Step 1800: Training loss 2.69920, Value loss 2.82372
Step 

Now to check what TrumpGPT has cooked up (Most likely very bad lol and not very much training due to hardware constraints)

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device) # Generate a context for the model to began to decode and generate
print(decode(m.generate(context, max_new_tokens=3000)[0].tolist())) # Here is the generation, increase the max tokens to get a longer string


"Ie in Namps! Bold Jast Hille Obage jouluse Dona So is I moup oncorese teding he's allional us really morning."
Gef with an misgets! Tike it for!
"Volion 2 Worpete --20urne."" Thanks @mibmcishtch Rorig so like it leater."
"""@2olinaJanaz:TrutifeNNFS"""
"""@Holfuun_quaker @unbeSJohn11: “@RotFaod @realDonaldTrump @Beffrian_feents Probauti http://t.co/xWQvb7oCG
"""@Ratiteo7Fin2Mons Twatcho/NBR38ATIvF46 NYFLL NAG: http://rYht6gY:/
"""@railynjTy6: @realDonaldTrump. I comme – but willion."""
"@gnya2016: Why @realDonaldTrump What You by Megin?
“Trump not.They!!! John, I ame sawing Mourty peeprive mose agree polloves.
@mannuma_mmin Christons Agenticons of Risery Noo with the I @sanids Mexand Time mis sechie Sappera great tis.
.@Penninikesude: @realDonaldTrump in keep is a great agroe a dedinatence up the Critist by that you tomorivievati: @AIS74QLKEF Stang Repora gresibout and noot sup cheasilys enfort-torry! #Pess#Cell Jeb StaIrate Hiliss in Doral, aboubt cisive to gain!"" Thanks.
It's liebe